In [172]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [173]:
IMAGE_SIZE = 227 
NUM_EPOCHS = 10
LR = 0.0001 # Learning Rate
MODEL_NAME = 'AlexNet'

DEVICE = ("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(DEVICE)

2.0.1+cpu
cpu


In [174]:
transform = transforms.Compose([

    transforms.Resize(256),
    transforms.RandomCrop(227),
   #transforms.Resize(227),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
   
])

In [175]:
train_data = pd.read_csv('./data/fashion-mnist_train.csv')
test_data = pd.read_csv('./data/fashion-mnist_test.csv')

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] # 총 10개의 클래스

# 6만개 데이터중 8천개만 학습에 사용
inputSize = 8000
train_data=train_data[:inputSize]

In [176]:
class FashionDataset(Dataset):
    def __init__(self, data, transform=None):        
        self.fashion_MNIST = list(data.values)
        self.transform = transform
        
        label, image = [], []
        
        for i in self.fashion_MNIST:
            label.append(i[0])
            image.append(i[1:])
        self.labels = np.asarray(label)
        self.images = np.asarray(image).reshape(-1, 28, 28).astype('float32')
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        image = self.images[idx]      
        
        if self.transform is not None:
            # 넘파이 배열을 transform 함수 실행 전에 PIL 이미지로 변환
            pil_image = Image.fromarray(np.uint8(image)) 
            image = self.transform(pil_image)
            
        return image, label

In [177]:
#train_loader = DataLoader(train_data, batch_size=64, shuffle=True, drop_last = True, pin_memory = True,num_workers=3)
                                                                                  # Tensor를 CPU -> GPU로 올림
#test_loader = DataLoader(test_data, batch_size=64, shuffle=True, drop_last = False, pin_memory = True,num_workers=3)

#FashionDataset클래스를이용해 csv파일에서 이미지로 변환, dataloader객체를 각각 train_loader, test_loader에 담음
train_loader = DataLoader(FashionDataset(train_data, transform=transform), batch_size=64, shuffle=True)
test_loader = DataLoader(FashionDataset(test_data, transform=transform), batch_size=64, shuffle=True)

In [178]:
class Alexnet(nn.Module):  
    def __init__(self):
        super().__init__()

        self.size = 5
        self.alpha = 1e-4
        self.beta = 0.75
        self.k = 2
        self.conv1 = nn.Sequential(
            # Input Channel (RGB: 3)(Grayscale: 1)
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0),# 227 -> 55
            nn.ReLU(),
            nn.LocalResponseNorm(size=self.size, alpha=self.alpha, beta=self.beta, k=self.k),
            nn.MaxPool2d(kernel_size=3, stride=2)# 55 -> 27
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.LocalResponseNorm(size=self.size, alpha=self.alpha, beta=self.beta, k=self.k),
            nn.MaxPool2d(3, 2) # 27 -> 13
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU()
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2) # 13 -> 6
        )

        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

   
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = out.view(out.size(0), -1)# 64x4096
        out = F.relu(self.fc1(out))
        out = F.dropout(out, 0.5)
        out = F.relu(self.fc2(out))
        out = F.dropout(out, 0.5)
        out = self.fc3(out)
        out = F.log_softmax(out, dim=1)

        return out

In [179]:
model = Alexnet().to(DEVICE)
criterion = F.nll_loss
error = nn.CrossEntropyLoss()#Multi-Class Classification 이기 때문에 Lossfunction으로 CrossEntropy 손실을 지정
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.NAdam(model.parameters()) #optimizer로 NAdam을사용
optimizer = optim.Adam(model.parameters())
import torchsummary
torchsummary.summary(model, input_size=(1, 227, 227), device=DEVICE)
#레이어 구조 출력

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          11,712
              ReLU-2           [-1, 96, 55, 55]               0
 LocalResponseNorm-3           [-1, 96, 55, 55]               0
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
 LocalResponseNorm-7          [-1, 256, 27, 27]               0
         MaxPool2d-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             ReLU-14          [-1, 256,

In [180]:
loss_list_train=[]
loss_list_val=[]
accuracy_train=[]

#학습함수
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        target = target.type(torch.LongTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()   
        accuracy_train.append((batch_idx / len(train_loader))*100)
        if (batch_idx + 1) % 30 == 0:
            print("Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [181]:
accuracy_val=[]
#test함수
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)  # loss之和除以data数量 -> mean
        accuracy_val.append(100. * correct / len(test_loader.dataset))
        print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
        print(test_loss)
        print(correct)
        print(accuracy_val)
        print('='*50)

In [182]:
num_epochs = 30
loss_list_train=[]
loss_list_val=[]
accuracy_train=[]
accuracy_val=[]
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


for epoch in range(num_epochs):
    print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            dataloader = train_loader
            model.train()  # Set model to training mode
        else:
            dataloader =test_loader
            model.eval()   # Set model to evaluate mode

        running_loss = 0.
        running_acc = 0.

            # Iterate over data.
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
            labels = labels.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

            optimizer.zero_grad()

                # forward and backward
            with torch.set_grad_enabled(phase == 'train'):
                preds = model(inputs)
                loss_value = error(preds, labels)
                preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                if phase == 'train':
                    loss_value.backward()
                    optimizer.step()

                # statistics
            running_loss += loss_value.item()
            running_acc += (preds_class == labels.data).float().mean()

        epoch_loss = running_loss / len(dataloader)
        epoch_acc = running_acc / len(dataloader)

        if phase == 'train':
            loss_list_train.append(epoch_loss)
            accuracy_train.append(epoch_acc*100)
        else:
            loss_list_val.append(epoch_loss)
            accuracy_val.append(epoch_acc*100)
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

Epoch 0/29:


100%|█████████████████████████████| 125/125 [01:55<00:00,  1.08it/s]

train Loss: 1.1789 Acc: 0.5465



100%|█████████████████████████████| 157/157 [00:50<00:00,  3.13it/s]

val Loss: 0.7050 Acc: 0.7358
Epoch 1/29:


...

val Loss: 0.3910 Acc: 0.8772
Epoch 29/29:



 ... (more hidden) ...

train Loss: 0.2291 Acc: 0.9137



 ... (more hidden) ...

val Loss: 0.4048 Acc: 0.8730
